# Notebook para aprendizado de mongo

Esse notebook serve unicamente para eu aprender as funções básicas do mongo antes de colocá-lo em um projeto, para isso farei um CRUD em uma collection, além de outras manipulações básicas.

Verifiquei que a cada requisição feita no banco ele conta como uma connection nova, deixando ali em aberto por um ou dois minutos. Devo melhorar isso para usar apenas uma conexão ao longo do notebook.

### Instalações e imports necessários
Talvez você precise reiniciar o kernel depois dessa ação

In [1]:
! pip install pymongo
! pip install dnspython
! pip install python-dotenv
from pymongo import MongoClient
import csv
from pprint import pprint
from dotenv import load_dotenv
import os

In [2]:
load_dotenv(dotenv_path='.env')

True

### Conexão

A conexão é feita usando as credenciais de um arquivo .env, eles são gerados da criação do cluster na cloud do MongoDB.

Já nas células seguintes ocorre a verificação das databases no cluster e o passo-a-passo da conexão com uma collection. Primeiro entra com o cliente, depois em um banco de dados dentro desse cliente e por fim no banco de dados é possível acessar a collection.

Salvei os dados que eu quero fazer upload em um csv. Nesse exato momento é 00h18 e eu estou sem criatividade, então essa coleção teste será sobre algo um tanto estranho: as letras do teclado do meu notebook. 

In [3]:
user = os.getenv('user')
password = os.getenv('password')
cluster = os.getenv('cluster')
url = 'mongodb+srv://'+user+':'+password+'@'+cluster+'.kivsk.gcp.mongodb.net/'

In [4]:
client = MongoClient(url)
cursor = client.list_databases()
for db in cursor:
    print(db)

{'name': 'notebook', 'sizeOnDisk': 8192.0, 'empty': False}
{'name': 'admin', 'sizeOnDisk': 294912.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 1445871616.0, 'empty': False}


In [5]:
db = client.get_database('notebook')
col = db.get_collection('teclado')

In [6]:
teclado = []
tecla = {}

with open('board_data.csv', 'r', encoding='utf-8-sig') as data:
    reader = csv.reader(data)
    keys = next(reader)
    for row in reader:
        tecla = {}
        for tup in zip(keys, row):
            if tup[1] != '':
                try:
                    tecla.update({tup[0]:float(tup[1])})
                except ValueError:
                    tecla.update({tup[0]:tup[1]})
        teclado.append(tecla)    

### Create
Primeiro me foi um tanto estranho ter duas funções diferentes para isso, mas até que faz sentido pra ser mais flexível dependendo do caso.

A primeira insere apenas um, enquanto que a segunda está inserindo todos os outros.

In [7]:
col.insert_one(
    teclado[0]
)

col.insert_many(
    teclado[1:len(teclado)]
)

### Read

In [8]:
# pega todos os documentos dessa coleção
cur = col.find({})
for doc in cur:
    print(doc)

{'_id': ObjectId('5f0a367036c2e66cc05bfe9c'), 'name': 'ze', 'type': 'letter', 'character': 'z', 'shift': 'Z', 'common_action': 'writes letter z', 'size_x': 1.0, 'size_y': 1.0}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9d'), 'name': 'xis', 'type': 'letter', 'character': 'x', 'shift': 'X', 'common_action': 'writes letter x', 'size_x': 1.0, 'size_y': 1.0}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9e'), 'name': 'ce', 'type': 'letter', 'character': 'c', 'shift': 'C', 'common_action': 'writes letter c', 'size_x': 1.0, 'size_y': 1.0}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9f'), 'name': 'comma', 'type': 'punctuation mark', 'character': ',', 'shift': '<', 'common_action': 'writes a comma', 'description': 'both characters on the left, comma lower than symbol of less than ', 'size_x': 1.0, 'size_y': 1.0}
{'_id': ObjectId('5f0a367036c2e66cc05bfea0'), 'name': 'shift', 'type': 'command', 'common_action': 'Change the size of the letters or use the top character', 'size_x': 3.0, 'size_y': 1.0}
{'_id': Ob

In [9]:
# só traz o primeiro, caso tenham mais com esse nome poderia ser um problema
cur = col.find({'name':'comma'})
pprint(cur.next())

{'_id': ObjectId('5f0a367036c2e66cc05bfe9f'),
 'character': ',',
 'common_action': 'writes a comma',
 'description': 'both characters on the left, comma lower than symbol of less '
                'than ',
 'name': 'comma',
 'shift': '<',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'punctuation mark'}


In [10]:
# $ne é um operador para 'not equal' existem outros que eu vou explorar depois já que parece algo bem básico e útil
cur = col.find(
    {'size_x': {'$ne': '1'}}
)
for doc in cur:
    pprint(doc)

{'_id': ObjectId('5f0a367036c2e66cc05bfe9c'),
 'character': 'z',
 'common_action': 'writes letter z',
 'name': 'ze',
 'shift': 'Z',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'letter'}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9d'),
 'character': 'x',
 'common_action': 'writes letter x',
 'name': 'xis',
 'shift': 'X',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'letter'}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9e'),
 'character': 'c',
 'common_action': 'writes letter c',
 'name': 'ce',
 'shift': 'C',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'letter'}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9f'),
 'character': ',',
 'common_action': 'writes a comma',
 'description': 'both characters on the left, comma lower than symbol of less '
                'than ',
 'name': 'comma',
 'shift': '<',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'punctuation mark'}
{'_id': ObjectId('5f0a367036c2e66cc05bfea0'),
 'common_action': 'Change the size of the letters or use the top character',
 'name': 'shift',
 'size_x'

### Update
Eu reparei que tinha medido o shift errado, na verdade (pelas proporções de uma tecla média + duas pequenas) a medida é de x é 2.6

In [11]:
col.update(
    {'name': 'shift'}, 
    {'$set':
     {'size_x': '2.6'}
    }
)

<ipython-input-11-74817acf9bda>:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  col.update(


{'n': 1,
 'nModified': 1,
 'opTime': {'ts': Timestamp(1594504911, 1), 't': 7},
 'electionId': ObjectId('7fffffff0000000000000007'),
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1594504911, 1),
  'signature': {'hash': b'\xa4\xbd\x9e\x83\x7fE\xcb S\xef>\x152vD\x95\xa46\xbbJ',
   'keyId': 6845058786030256131}},
 'operationTime': Timestamp(1594504911, 1),
 'updatedExisting': True}

In [12]:
cur = col.find({'name': 'shift'})
pprint(cur.next())

{'_id': ObjectId('5f0a367036c2e66cc05bfea0'),
 'common_action': 'Change the size of the letters or use the top character',
 'name': 'shift',
 'size_x': '2.6',
 'size_y': 1.0,
 'type': 'command'}


### Delete

In [13]:
#apesar de haver dois com type command apenas o primeiro é excluído

col.delete_one({'character':'z'})
col.delete_one({'character':'x'})
col.delete_one({'type':'command'})
cur = col.find({})
for doc in cur:
    pprint(doc)

{'_id': ObjectId('5f0a367036c2e66cc05bfe9e'),
 'character': 'c',
 'common_action': 'writes letter c',
 'name': 'ce',
 'shift': 'C',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'letter'}
{'_id': ObjectId('5f0a367036c2e66cc05bfe9f'),
 'character': ',',
 'common_action': 'writes a comma',
 'description': 'both characters on the left, comma lower than symbol of less '
                'than ',
 'name': 'comma',
 'shift': '<',
 'size_x': 1.0,
 'size_y': 1.0,
 'type': 'punctuation mark'}
{'_id': ObjectId('5f0a367036c2e66cc05bfea1'),
 'common_action': 'ask for help',
 'description': 'no-sound icon with F1 writen on down-right, top-right has a '
                'led light that turns red when wound is off',
 'function': 'turns on/off the sound',
 'name': 'f1',
 'size_x': 0.8,
 'size_y': 0.5,
 'type': 'command'}


In [14]:
# no caso como não há nada nos brackets isso será praticamente um drop
col.delete_many({})
cur = col.find({})
for doc in cur:
    pprint(doc)

### Das ist alles Leute

In [15]:
client.close()